# Out-of-State-Contributions: National Analysis

How much out-of-state money have candidates nationally raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [49]:
from functools import reduce
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


Import contributions data.

In [50]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/joe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,12,13,14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6051953 entries, 0 to 6051952
Data columns (total 18 columns):
candidate                 object
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(1), int64(1), object(16)
memory usage: 831.1+ MB


<IPython.core.display.Javascript object>

Convert the contribution date and latest month columns to datetime data type and the redistricting role column to string (object) datatype.

In [51]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions["redistricting_role"] = contributions["redistricting_role"].astype(object)
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6051953 entries, 0 to 6051952
Data columns (total 18 columns):
candidate                 object
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(1), object(14)
memory usage: 831.1+ MB


## Calculate out-of-state contributions by party and year

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [52]:
contributions_by_party = contributions.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"454,991,330.51"
1,2010,Democratic,out-of-state,"56,810,108.85"
2,2010,Democratic,unknown,"1,312,244.72"
3,2010,Nonpartisan,in-state,"942,676.00"
4,2010,Nonpartisan,out-of-state,"136,753.70"
5,2010,Nonpartisan,unknown,"-19,688.72"
6,2010,Republican,in-state,"631,986,359.03"
7,2010,Republican,out-of-state,"48,159,238.63"
8,2010,Republican,unknown,"627,736.24"
9,2010,Third-Party,in-state,"5,257,658.51"


Pivot dataframe to aggregate each party's data in a single row.

In [53]:
contributions_by_party = pd.pivot_table(contributions_by_party, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party

party         amount                             \
year                                2010                              
in_out_state                    in-state  out-of-state      unknown   
0              Democratic 454,991,330.51 56,810,108.85 1,312,244.72   
1             Nonpartisan     942,676.00    136,753.70   -19,688.72   
2              Republican 631,986,359.03 48,159,238.63   627,736.24   
3             Third-Party   5,257,658.51    607,163.33 1,531,169.31   
4                 Unknown            nan           nan          nan   

                                                                        \
year                   2014                                       2018   
in_out_state       in-state  out-of-state       unknown       in-state   
0            380,863,502.49 65,669,233.08  5,969,356.57 593,342,876.83   
1              1,672,746.69    156,743.05      4,068.00   1,376,332.38   
2            445,191,983.93 72,850,170.20 10,986,718.52 700,498,617.20   
3              9,019,459.94    533,565.54     37,435.40   3,810,315.64   
4                       nan           nan           nan      28,042.01   

                                         
year                                     
in_out_state  out-of-state      unknown  
0            87,513,130.10 8,089,220.32  
1               310,506.30    -3,156.23  
2            66,420,967.50 8,863,713.00  
3               340,839.00   114,731.23  
4                 7,944.09          nan

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [54]:
contributions_by_party.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [55]:
contributions_by_party.columns = ["party", "18_in_state", "18_out_of_state", "18_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "10_in_state", "10_out_of_state", "10_unknown"]
contributions_by_party

,party,18_in_state,18_out_of_state,18_unknown,14_in_state,14_out_of_state,14_unknown,10_in_state,10_out_of_state,10_unknown
0,Democratic,"454,991,330.51","56,810,108.85","1,312,244.72","380,863,502.49","65,669,233.08","5,969,356.57","593,342,876.83","87,513,130.10","8,089,220.32"
1,Nonpartisan,"942,676.00","136,753.70","-19,688.72","1,672,746.69","156,743.05","4,068.00","1,376,332.38","310,506.30","-3,156.23"
2,Republican,"631,986,359.03","48,159,238.63","627,736.24","445,191,983.93","72,850,170.20","10,986,718.52","700,498,617.20","66,420,967.50","8,863,713.00"
3,Third-Party,"5,257,658.51","607,163.33","1,531,169.31","9,019,459.94","533,565.54","37,435.40","3,810,315.64","340,839.00","114,731.23"
4,Unknown,0.00,0.00,0.00,0.00,0.00,0.00,"28,042.01","7,944.09",0.00


Calculate the total contributions by cycle.

In [56]:
contributions_by_party["18_total"] = contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"]
contributions_by_party["14_total"] = contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"]
contributions_by_party["10_total"] = contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"]
contributions_by_party = contributions_by_party[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [57]:
contributions_by_party["pct_18_in_state"] = contributions_by_party["18_in_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_out_of_state"] = contributions_by_party["18_out_of_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_unknown"] = contributions_by_party["18_unknown"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_14_in_state"] = contributions_by_party["14_in_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_out_of_state"] = contributions_by_party["14_out_of_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_unknown"] = contributions_by_party["10_unknown"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_10_in_state"] = contributions_by_party["10_in_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_out_of_state"] = contributions_by_party["14_out_of_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_unknown"] = contributions_by_party["10_unknown"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"454,991,330.51","56,810,108.85","1,312,244.72","513,113,684.08","380,863,502.49","65,669,233.08","5,969,356.57","452,502,092.14","593,342,876.83","87,513,130.10","8,089,220.32","688,945,227.25",0.89,0.11,0.00,0.84,0.15,0.02,0.86,0.10,0.01
1,Nonpartisan,"942,676.00","136,753.70","-19,688.72","1,059,740.98","1,672,746.69","156,743.05","4,068.00","1,833,557.74","1,376,332.38","310,506.30","-3,156.23","1,683,682.45",0.89,0.13,-0.02,0.91,0.09,-0.00,0.82,0.09,-0.00
2,Republican,"631,986,359.03","48,159,238.63","627,736.24","680,773,333.90","445,191,983.93","72,850,170.20","10,986,718.52","529,028,872.65","700,498,617.20","66,420,967.50","8,863,713.00","775,783,297.70",0.93,0.07,0.00,0.84,0.14,0.02,0.90,0.09,0.01
3,Third-Party,"5,257,658.51","607,163.33","1,531,169.31","7,395,991.15","9,019,459.94","533,565.54","37,435.40","9,590,460.88","3,810,315.64","340,839.00","114,731.23","4,265,885.87",0.71,0.08,0.21,0.94,0.06,0.01,0.89,0.13,0.03
4,Unknown,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"28,042.01","7,944.09",0.00,"35,986.10",nan,nan,nan,nan,nan,nan,0.78,0.00,0.00


## Calculate 2018 out-of-state contributions by redistricting role

Filter the contributions data to the 2018 cycle.

In [58]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2275012 entries, 0 to 2275011
Data columns (total 18 columns):
candidate                 object
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(1), object(14)
memory usage: 329.8+ MB


Group by redistricting role and in-vs.-out-of-state contribution status and sum contributions.

In [59]:
contributions_by_redistricting = contributions.groupby(["redistricting_role", "in_out_state"])["amount"].sum().reset_index()
contributions_by_redistricting

,redistricting_role,in_out_state,amount
0,N,in-state,"222,189,963.25"
1,N,out-of-state,"27,213,022.74"
2,N,unknown,"4,360,094.74"
3,Y,in-state,"1,076,866,220.81"
4,Y,out-of-state,"127,380,364.25"
5,Y,unknown,"12,704,413.58"


Pivot dataframe to aggregate each role's data in a single row.

In [62]:
contributions_by_redistricting = pd.pivot_table(contributions_by_redistricting, index=["redistricting_role"], columns=["in_out_state"]).reset_index()
contributions_by_redistricting

redistricting_role           amount                             
in_out_state                            in-state   out-of-state       unknown
0                             N   222,189,963.25  27,213,022.74  4,360,094.74
1                             Y 1,076,866,220.81 127,380,364.25 12,704,413.58

Flatten the resulting dataframe's multi-index columns.

In [63]:
contributions_by_redistricting.columns = ["redistricting_role", "in_state", "out_of_state", "unknown"]
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown
0,N,"222,189,963.25","27,213,022.74","4,360,094.74"
1,Y,"1,076,866,220.81","127,380,364.25","12,704,413.58"


Calculate the total contributions by redistricting role.

In [64]:
contributions_by_redistricting["total"] = contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions by redistricting role.

In [65]:
contributions_by_redistricting["pct_in_state"] = contributions_by_redistricting["in_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_out_of_state"] = contributions_by_redistricting["out_of_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_unknown"] = contributions_by_redistricting["unknown"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown,total,pct_in_state,pct_out_of_state,pct_unknown
0,N,"222,189,963.25","27,213,022.74","4,360,094.74","253,763,080.73",0.88,0.11,0.02
1,Y,"1,076,866,220.81","127,380,364.25","12,704,413.58","1,216,950,998.64",0.88,0.10,0.01


## Calculate contributions by candidate status and year

Group by year and candidate status and in-vs.-out-of-state contribution status and sum contributions.

In [41]:
contributions_by_status = contributions.groupby(["year", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_status

,year,standardized_status,in_out_state,amount
0,2010,ADVANCED TO GENERAL,in-state,"854,672,580.19"
1,2010,ADVANCED TO GENERAL,out-of-state,"89,236,961.16"
2,2010,ADVANCED TO GENERAL,unknown,"2,732,449.53"
3,2010,DID NOT ADVANCE,in-state,"238,505,443.86"
4,2010,DID NOT ADVANCE,out-of-state,"16,476,303.35"
5,2010,DID NOT ADVANCE,unknown,"719,012.02"
6,2014,ADVANCED TO GENERAL,in-state,"710,162,108.85"
7,2014,ADVANCED TO GENERAL,out-of-state,"120,696,904.53"
8,2014,ADVANCED TO GENERAL,unknown,"11,732,958.70"
9,2014,DID NOT ADVANCE,in-state,"126,585,584.20"


Pivot dataframe to aggregate each candidate status' data in a single row.

In [42]:
contributions_by_status = pd.pivot_table(contributions_by_status, index=["standardized_status"], columns=["year", "in_out_state"]).reset_index()
contributions_by_status

standardized_status         amount                             \
year                                        2010                              
in_out_state                            in-state  out-of-state      unknown   
0             ADVANCED TO GENERAL 854,672,580.19 89,236,961.16 2,732,449.53   
1                 DID NOT ADVANCE 238,505,443.86 16,476,303.35   719,012.02   

                                                                         \
year                   2014                                        2018   
in_out_state       in-state   out-of-state       unknown       in-state   
0            710,162,108.85 120,696,904.53 11,732,958.70 929,935,372.71   
1            126,585,584.20  18,512,807.34  5,264,619.79 369,120,811.35   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            114,551,238.31 11,040,180.78  
1             40,042,148.68  6,024,327.54

Flatten the resulting dataframe's multi-index columns.

In [44]:
contributions_by_status.columns = ["standardized_status", "18_in_state", "18_out_of_state", "18_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "10_in_state", "10_out_of_state", "10_unknown"]
contributions_by_status

,standardized_status,18_in_state,18_out_of_state,18_unknown,14_in_state,14_out_of_state,14_unknown,10_in_state,10_out_of_state,10_unknown
0,ADVANCED TO GENERAL,"854,672,580.19","89,236,961.16","2,732,449.53","710,162,108.85","120,696,904.53","11,732,958.70","929,935,372.71","114,551,238.31","11,040,180.78"
1,DID NOT ADVANCE,"238,505,443.86","16,476,303.35","719,012.02","126,585,584.20","18,512,807.34","5,264,619.79","369,120,811.35","40,042,148.68","6,024,327.54"


Calculate the total contributions by cycle.

In [46]:
contributions_by_status["18_total"] = contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"]
contributions_by_status["14_total"] = contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"]
contributions_by_status["10_total"] = contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"]
contributions_by_status = contributions_by_status[["standardized_status", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [47]:
contributions_by_status["pct_18_in_state"] = contributions_by_status["18_in_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_out_of_state"] = contributions_by_status["18_out_of_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_unknown"] = contributions_by_status["18_unknown"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_14_in_state"] = contributions_by_status["14_in_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_out_of_state"] = contributions_by_status["14_out_of_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_unknown"] = contributions_by_status["10_unknown"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_10_in_state"] = contributions_by_status["10_in_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_out_of_state"] = contributions_by_status["14_out_of_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_unknown"] = contributions_by_status["10_unknown"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,standardized_status,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,ADVANCED TO GENERAL,"854,672,580.19","89,236,961.16","2,732,449.53","946,641,990.88","710,162,108.85","120,696,904.53","11,732,958.70","842,591,972.08","929,935,372.71","114,551,238.31","11,040,180.78","1,055,526,791.80",0.90,0.09,0.00,0.84,0.14,0.01,0.88,0.11,0.01
1,DID NOT ADVANCE,"238,505,443.86","16,476,303.35","719,012.02","255,700,759.23","126,585,584.20","18,512,807.34","5,264,619.79","150,363,011.33","369,120,811.35","40,042,148.68","6,024,327.54","415,187,287.57",0.93,0.06,0.00,0.84,0.12,0.04,0.89,0.04,0.01


## Export the data

In [66]:
writer = pd.ExcelWriter("data/national_analysis.xlsx")
contributions_by_party.to_excel(writer, "contributions_by_party", index=False)
contributions_by_redistricting.to_excel(writer, "contributions_by_redistricting", index=False)
contributions_by_status.to_excel(writer, "contributions_by_status", index=False)
writer.save()